In [3]:
import sys
import json
import random
sys.version

'3.7.4 (default, Oct 15 2019, 22:29:14) \n[GCC 7.4.0]'

In [6]:
from py2neo import Graph
from neo4j import GraphDatabase
NEO4J_URI="bolt://localhost:7687"
SC_JSON = "semantic-scholar-raw/s2-corpus-slim.json"
SC_JSON_GITHUB = "https://raw.githubusercontent.com/diegoquintanav/od-lab-1/master/notebooks/semantic-scholar-raw/s2-corpus-slim.json"

graph = Graph(NEO4J_URI)
gdb = GraphDatabase.driver(uri=NEO4J_URI, auth=None)

In [7]:
from faker import Faker
from faker.providers import lorem
fake = Faker()
fake.seed_instance(42)
fake.add_provider(lorem)

http://s2-public-api-prod.us-west-2.elasticbeanstalk.com/corpus/

Download a part of the corpus with the `aws` command line interface, like

```aws s3 cp --no-sign-request s3://ai2-s2-research-public/open-corpus/2018-05-03/s2-corpus-00.gz .```

or

```wget https://s3-us-west-2.amazonaws.com/ai2-s2-research-public/open-corpus/2018-05-03/s2-corpus-00.gz```

And then `gunzip` it.

In [31]:
!ls semantic-scholar-raw/

license.txt   s2-corpus-00	    s2-corpus-clean.tmp
manifest.txt  s2-corpus-clean.json  s2-corpus-slim.json


In [32]:
!wc -l semantic-scholar-raw/s2-corpus-00

1000000 semantic-scholar-raw/s2-corpus-00


Filter out those lines that have empty entities, empty inCitations, and empty outCitations

In [23]:
!grep -v -e '"entities":\[\]' -e '"inCitations":\[\]' -e '"outCitations":\[\]' semantic-scholar-raw/s2-corpus-00 > semantic-scholar-raw/s2-corpus-clean.tmp
!wc -l semantic-scholar-raw/s2-corpus-clean.tmp

212901 semantic-scholar-raw/s2-corpus-clean.tmp


Filter out those rows with empty attributes

In [28]:
!grep -v -e '"journalName":""' -e '"title":""' -e '"journalVolume":""' -e '"id":""' -e '"ids":""' -e '"doi":""' -e '"doiUrl":""' semantic-scholar-raw/s2-corpus-clean.tmp > semantic-scholar-raw/s2-corpus-clean.json
!wc -l semantic-scholar-raw/s2-corpus-clean.json

86609 semantic-scholar-raw/s2-corpus-clean.json


Obtain a smaller sample from that data

In [29]:
!head -100 semantic-scholar-raw/s2-corpus-clean.json > semantic-scholar-raw/s2-corpus-slim.json

In [27]:
!head -1 semantic-scholar-raw/s2-corpus-slim.json | jq

{
  "entities": [
    "Eating",
    "Ethanol",
    "Isopropyl Alcohol 0.7 ML/ML Medicated Pad"
  ],
  "journalVolume": "31 4",
  "journalPages": "385-8",
  "pmid": "8879286v1",
  "year": 1996,
  "outCitations": [
    "40bb6636803d10626bfafa3dbf940fdac903d11c",
    "18ffb4ae9e0d8f0c43342c4c6b993f3ce34bd579",
    "827b793fbc923d3e068b2008dc02bb63e7069fb9",
    "b96cb6dd0bba1e9928eb6315e18339f667c6799f",
    "48431576480666445bdcb112533aa885d4ac1ad1",
    "1380d901dbce2dca8eba90501179485d4fe1e5d4",
    "4203ad4dc8530a92473e5576650eca84b51c9eee",
    "96b85a405b002e11199f967280ec9a9736543c86",
    "8d72d01b1b670c7bb65408e91d1705f0c95b1174",
    "7842615744a118a2d4a239787e9450c26c98c9f9",
    "6ed7439f9537fce3f82144843622b9c670a95d7e",
    "55787878e91f6efdf14e16438e733d80581f3bec",
    "e28468ccfa0ddce1d060d39d25305f6abab2eced",
    "5f6ba13f126408d25849c518bddf4812281ae970"
  ],
  "s2Url": "https://semanticscholar.org/paper/30aaf92bde9bf6cdc3dff237691fd5db55d4cb4a",
  "s2PdfUrl": "http://

The final modified JSON is accesible now from github

In [72]:
myjson = "https://raw.githubusercontent.com/diegoquintanav/od-lab-1/master/notebooks/semantic-scholar-raw/s2-corpus-slim.json"

## A.2 Loading

In [165]:
graph.run("MATCH (n) DETACH DELETE n").stats()

constraints_added: 0
constraints_removed: 0
contained_updates: False
indexes_added: 0
indexes_removed: 0
labels_added: 0
labels_removed: 0
nodes_created: 0
nodes_deleted: 0
properties_set: 0
relationships_created: 0
relationships_deleted: 0

We will be using apoc (https://github.com/neo4j-contrib/neo4j-apoc-procedures)

In [166]:
graph.run("RETURN apoc.version();").data()

[{'apoc.version()': '3.5.0.9'}]

In [167]:
graph.run("call dbms.components() yield name, versions, edition unwind versions as version return name, version, edition;").data()

[{'name': 'Neo4j Kernel', 'version': '3.5.15', 'edition': 'enterprise'}]

In [168]:
query = """
WITH $myjson AS url 
CALL apoc.load.json(url) 
YIELD value 
RETURN value 
LIMIT 1"""

graph.run(query, myjson = myjson).data()

[{'value': {'doiUrl': 'https://doi.org/10.1007/BF02270828',
   'venue': 'Evolutionary Ecology',
   'journalName': 'Evolutionary Ecology',
   'sources': [],
   'year': 2005,
   'paperAbstract': 'I compare the relationship between bill size (depth) and body size among different taxa of seed-eating finches to test the hypothesis (Schluter, 1988a) that in habitats where seed-eating finches are vulnerable to predators, finches have larger bodies relative to their bill size. In support of this hypothesis, ground-foraging finches on continents (Emberizidae, Passeridae, Fringillidae), where predators are more common, have larger bodies relative to their bill sizes than do ground-foraging finches on islands (Emberizidae, Fringillidae). Ground-feeding finches on continents may also be more vulnerable to predators than tree- and shrub-foraging finches. As predicted, in North America, ground-feeding finches (Emberizinae) have larger bodies relative to their bill size than do tree- and shrub-foragi

Here, `"myjson"` is an alias for the path to the file, as defined in https://neo4j.com/docs/labs/apoc/current/import/load-json/

### create Article instances

In [169]:
query_apoc = """WITH $myjson AS url 
CALL apoc.load.json(url) 
YIELD value as row
MERGE (art:Article {
        title: row.title,
        id: row.id, doi: 
        row.doi, 
        doi_url:row.doiUrl,
        year: row.year})
MERGE (jour:Journal {
        name: row.journalName,
        volume: row.journalVolume,
        pages: row.journalPages})-[:PUBLISHED_IN]->(art)
FOREACH (author in row.authors |
    MERGE (auth:Author {name: author.name, id: author.ids})-[:AUTHORS]->(art)
)
FOREACH (k in row.entities |
    MERGE (art)-[:CONTAINS]->(kw: Keyword {name: k}))"""

graph.run(query_apoc, myjson = myjson).stats()

constraints_added: 0
constraints_removed: 0
contained_updates: True
indexes_added: 0
indexes_removed: 0
labels_added: 1898
labels_removed: 0
nodes_created: 1898
nodes_deleted: 0
properties_set: 2989
relationships_created: 1798
relationships_deleted: 0

Some data is left to be simulated

### Faking citations

In [170]:
q_article_ids = "MATCH (n:Article) RETURN n.id"
article_ids = [v["n.id"] for v in graph.run(q_article_ids).data()]
n_articles = len(article_ids)
article_ids[:5]

['b87d1a177d07a8e7d66be4c9fb70ea0a03154772',
 '00a234d1634e2335778b40aeacf0482ad5e2d70f',
 '31ecf94c75026cf168d6db275fcecad83fbbe2f4',
 '02fc5fada3a31fc2b128b4784fd2e8c8a75b8b08',
 '8fb9c95bf34a0f28dc05819cb4aada0cb94fe555']

In [171]:
# https://neo4j.com/docs/driver-manual/1.7/sessions-transactions/#driver-transactions-transaction-functions
q_add_citation_rel_id = """MATCH (a:Article),(b:Article)
WHERE a.id = $id_a AND b.id = $id_b
MERGE p=(a)-[r:CITED_BY]->(b)
RETURN p"""

def add_citation_rel(driver, id_a, id_b):
    with driver.session() as session:
        # Caller for transactional unit of work
        return session.write_transaction(create_citation_rel, id_a, id_b)

# Simple implementation of the unit of work
def create_citation_rel(tx, id_a, id_b):
    return tx.run(q_add_citation_rel_id, id_a=id_a, id_b = id_b)

Add 500 relationships of type `cited_by`

In [172]:
random.seed(42)
# pick a sample of 500 papers, and make them be cited by other three papers at random
created_citations = []

for article in article_ids:
    for citation in random.sample(article_ids, random.randint(5, 50)):
        if article != citation: # they can't cite themselves
            _out = add_citation_rel(gdb, id_a=article, id_b=citation)
            created_citations.append(_out.data())
        else:
            print("Skipping: ", article, citation)

created_citations[:5]

Skipping:  b87d1a177d07a8e7d66be4c9fb70ea0a03154772 b87d1a177d07a8e7d66be4c9fb70ea0a03154772
Skipping:  c4062742b4e0d13cfa0e992fdf2cebf2eb71c415 c4062742b4e0d13cfa0e992fdf2cebf2eb71c415
Skipping:  b95193355f5e3823461f228d9b2b1ad2d7ffc1fe b95193355f5e3823461f228d9b2b1ad2d7ffc1fe
Skipping:  421da3edd7d840ce58b811d2ad62dac3dafc7663 421da3edd7d840ce58b811d2ad62dac3dafc7663
Skipping:  b1b12644378d1e61f298e247ad936f0a16bb9989 b1b12644378d1e61f298e247ad936f0a16bb9989
Skipping:  1799f8fd782a5028eeccbed2daa6959b08b66308 1799f8fd782a5028eeccbed2daa6959b08b66308
Skipping:  40ee9c895397af6326cbcf97f6448c8419940074 40ee9c895397af6326cbcf97f6448c8419940074
Skipping:  25aa00f2fd8ca7dfbafb35d6d88fc3189e649c2f 25aa00f2fd8ca7dfbafb35d6d88fc3189e649c2f
Skipping:  ea4d7b1168128acbd3c4f2a990fae09a7526a6d2 ea4d7b1168128acbd3c4f2a990fae09a7526a6d2
Skipping:  7a69f0020f6131fb0fd82d7ae3c30300c694c8b4 7a69f0020f6131fb0fd82d7ae3c30300c694c8b4
Skipping:  3dffad910b096a2065f292ace889567ba219d40a 3dffad910b096a2065

[[{'p': <Path start=<Node id=46 labels={'Article'} properties={'doi_url': 'https://doi.org/10.3978/j.issn.2305-5839.2014.02.10', 'id': 'b87d1a177d07a8e7d66be4c9fb70ea0a03154772', 'title': 'Pulmonary carcinosarcoma with heterologous component: report of two cases with literature review.', 'year': 2014, 'doi': '10.3978/j.issn.2305-5839.2014.02.10'}> end=<Node id=243 labels={'Article'} properties={'doi_url': 'https://doi.org/10.1016/j.engappai.2008.04.018', 'id': '2c86deb0e77a7fb9853472ec39749d209adb9440', 'title': 'Automatic semantic indexing of medical images using a web ontology language for case-based image retrieval', 'year': 2009, 'doi': '10.1016/j.engappai.2008.04.018'}> size=1>}],
 [{'p': <Path start=<Node id=46 labels={'Article'} properties={'doi_url': 'https://doi.org/10.3978/j.issn.2305-5839.2014.02.10', 'id': 'b87d1a177d07a8e7d66be4c9fb70ea0a03154772', 'title': 'Pulmonary carcinosarcoma with heterologous component: report of two cases with literature review.', 'year': 2014, 'd

### Faking reviewers 

We will create a `REVIEWS` relationship between authors and articles

In [173]:
# delete relationships
graph.run("MATCH p=(:Author)-[r:REVIEWS]->(:Article) DELETE r RETURN COUNT(r)").stats()

constraints_added: 0
constraints_removed: 0
contained_updates: False
indexes_added: 0
indexes_removed: 0
labels_added: 0
labels_removed: 0
nodes_created: 0
nodes_deleted: 0
properties_set: 0
relationships_created: 0
relationships_deleted: 0

In [174]:
q_author_ids = "MATCH (n:Author) RETURN n.id"
author_ids = [v["n.id"] for v in graph.run(q_author_ids).data()]    
author_ids[:5]

[['3902390'], ['13413837'], ['24148377'], ['1792216'], ['2181642']]

#### Assign reviewers randomly to 1000 articles


In [175]:
# assign reviewers randomly to 1000 articles
random.seed(42)

q_add_reviewers = """MATCH (a:Article),(b:Author)
WHERE a.id = $article_id AND b.id = $author_id
AND NOT (b)-[:AUTHORS]->(a)
MERGE p=(b)-[:REVIEWS]->(a)
RETURN p"""

created_review_rel = []

with gdb.session() as session:
    for article_id in article_ids:
        for author_id in random.sample(author_ids, random.choice([2, 6])):
            _out = graph.run(q_add_reviewers, article_id=article_id, author_id=author_id)
            created_review_rel.append(_out.data())
            
created_review_rel[:5]

[[{'p': (Samuel C Mok)-[:REVIEWS {}]->(_46)}],
 [{'p': (Shaowei Ding)-[:REVIEWS {}]->(_46)}],
 [{'p': (Donald E Fry)-[:REVIEWS {}]->(_65)}],
 [{'p': (Beatriz Brignani)-[:REVIEWS {}]->(_65)}],
 [{'p': (Martin Kanning)-[:REVIEWS {}]->(_65)}]]

### Fake conferences and editions

In [176]:
# delete conference and editions
graph.run("MATCH (n:Conference) DETACH DELETE n RETURN COUNT(n) as deleted_conferences").data()

[{'deleted_conferences': 0}]

In [177]:
graph.run("MATCH (n:Edition) DETACH DELETE n RETURN COUNT(n) as deleted_editions").data()


[{'deleted_editions': 0}]

We will fake 600 editions, distributed between 100 conferences

In [178]:
random.seed(42)
N_CONFERENCES = 100

fake_conferences = [{
    "id": idx, 
    "name": fake.name()}
    for idx, _ in enumerate(range(100), start=1)]

fake_editions = [
    {
        "id": idx, 
        "venue": fake.name(),
        "city": fake.city(),
        "year": fake.year()}
    for idx, _ in enumerate(range(600), start=1)]

# assign 4 to 20 editions to each conference
for conference in fake_conferences:
    conference["editions"] = []
    for edition in random.sample(fake_editions, random.randint(4, 20)):
        conference["editions"].append(edition)

fake_conferences[:2]

[{'id': 1,
  'name': 'Leslie Wilson',
  'editions': [{'id': 26,
    'venue': 'Mr. Ryan Adams',
    'city': 'Lyonsside',
    'year': '2003'},
   {'id': 282,
    'venue': 'Shannon Mcdaniel',
    'city': 'New Jordan',
    'year': '1989'},
   {'id': 251,
    'venue': 'Victoria Lopez',
    'city': 'Cherylville',
    'year': '2015'},
   {'id': 229, 'venue': 'Juan Anderson', 'city': 'Perezberg', 'year': '2010'},
   {'id': 143, 'venue': 'Brittney Burns', 'city': 'Meganfurt', 'year': '2012'},
   {'id': 105, 'venue': 'Amber Palmer', 'city': 'Romerotown', 'year': '1971'},
   {'id': 559, 'venue': 'Susan Taylor', 'city': 'Mooreview', 'year': '1988'}]},
 {'id': 2,
  'name': 'Brian Murphy',
  'editions': [{'id': 433,
    'venue': 'Andrea Burke',
    'city': 'South Scott',
    'year': '1976'},
   {'id': 33,
    'venue': 'Matthew Gentry',
    'city': 'Carpenterberg',
    'year': '2011'},
   {'id': 31, 'venue': 'Eric Reid', 'city': 'North Nichole', 'year': '1982'},
   {'id': 96,
    'venue': 'Jennifer S

In [179]:
q_create_conference = "MERGE (c:Conference {id: $cid, name: $name}) RETURN c"
q_create_edition = "MERGE (e:Edition {id: $eid, conference_id: $cid, venue: $venue, city: $city, year: $year}) RETURN e"


q_merge_conference_and_edition = """MATCH (c:Conference), (e:Edition)
WHERE c.id = $cid AND e.id = $eid
MERGE p=(c)-[:IN]->(e)
RETURN p"""

created_conferences = []

with gdb.session() as session:
    for conference in fake_conferences:
        # create conference
        session.run(q_create_conference, cid = conference["id"], name = conference["name"])
        for edition in conference["editions"]:
            # create edition
            session.run(q_create_edition,
                        eid = edition["id"],
                        cid = conference["id"], 
                        venue = edition["venue"], 
                        city = edition["city"], 
                        year = edition["year"])
            # assign it to conference
            _out = session.run(q_merge_conference_and_edition, eid = edition["id"], cid = conference["id"])
            created_conferences.append(_out.data())

In [180]:
created_conferences[:5]

[[{'p': <Path start=<Node id=8085 labels={'Conference'} properties={'name': 'Leslie Wilson', 'id': 1}> end=<Node id=8086 labels={'Edition'} properties={'venue': 'Mr. Ryan Adams', 'conference_id': 1, 'id': 26, 'city': 'Lyonsside', 'year': '2003'}> size=1>}],
 [{'p': <Path start=<Node id=8085 labels={'Conference'} properties={'name': 'Leslie Wilson', 'id': 1}> end=<Node id=8057 labels={'Edition'} properties={'venue': 'Shannon Mcdaniel', 'conference_id': 1, 'id': 282, 'city': 'New Jordan', 'year': '1989'}> size=1>}],
 [{'p': <Path start=<Node id=8085 labels={'Conference'} properties={'name': 'Leslie Wilson', 'id': 1}> end=<Node id=8058 labels={'Edition'} properties={'venue': 'Victoria Lopez', 'conference_id': 1, 'id': 251, 'city': 'Cherylville', 'year': '2015'}> size=1>}],
 [{'p': <Path start=<Node id=8085 labels={'Conference'} properties={'name': 'Leslie Wilson', 'id': 1}> end=<Node id=8060 labels={'Edition'} properties={'venue': 'Juan Anderson', 'conference_id': 1, 'id': 229, 'city': 'P

### Assigning articles to editions

And we will assign articles randomly to 2 to 10 editions

In [181]:
fake_editions[:1]

[{'id': 1, 'venue': 'Monica Carter', 'city': 'Stephanietown', 'year': '1986'}]

In [182]:
# assign each paper to 2-10 editions
random.seed(42)

q_link_to_conf = """MATCH (a:Article), (b:Edition)
WHERE a.id = $aid AND b.id = $eid
MERGE p=(a)-[r:PUBLISHED_IN]->(b)
RETURN p"""

created_art_ed_rel = []

with gdb.session() as session:
    for aid in article_ids:
        for edition in random.sample(fake_editions, random.randint(2, 10)):
            _out = session.run(q_link_to_conf, aid = aid, eid = edition["id"])
            created_art_ed_rel.append(_out.data())
            
created_art_ed_rel[:1]

[[{'p': <Path start=<Node id=46 labels={'Article'} properties={'doi_url': 'https://doi.org/10.3978/j.issn.2305-5839.2014.02.10', 'id': 'b87d1a177d07a8e7d66be4c9fb70ea0a03154772', 'title': 'Pulmonary carcinosarcoma with heterologous component: report of two cases with literature review.', 'year': 2014, 'doi': '10.3978/j.issn.2305-5839.2014.02.10'}> end=<Node id=8086 labels={'Edition'} properties={'venue': 'Mr. Ryan Adams', 'conference_id': 1, 'id': 26, 'city': 'Lyonsside', 'year': '2003'}> size=1>},
  {'p': <Path start=<Node id=46 labels={'Article'} properties={'doi_url': 'https://doi.org/10.3978/j.issn.2305-5839.2014.02.10', 'id': 'b87d1a177d07a8e7d66be4c9fb70ea0a03154772', 'title': 'Pulmonary carcinosarcoma with heterologous component: report of two cases with literature review.', 'year': 2014, 'doi': '10.3978/j.issn.2305-5839.2014.02.10'}> end=<Node id=8711 labels={'Edition'} properties={'venue': 'Mr. Ryan Adams', 'conference_id': 33, 'id': 26, 'city': 'Lyonsside', 'year': '2003'}> s

## A.3 Evolving the graph 



### Store the review and the approval sent by each reviewer

Since the queries in part B don't need this information, we can store the `review_decision` and the `review_text` as _edge_ attributes of the relation `REVIEWED_BY`.


Alternatively, and in the case these reviews were a requirement in part B, they can be implemented by creating a `Review` node, and linking it to articles, in a four step process.

1. Create a `Review` article for each `Article` that has reviewers (i.e. is connected to authors by `reviewed_by` edges). Each `Review` node will have an attribute named `review_contents´.
2. Attach review to the article
3. Attach reviewers to the review instance
4. Optional, delete the `reviewed_by` edges from the step 1.

In [184]:
with gdb.session() as session:
    out = session.run("MATCH p=(:Author)-[r:REVIEWS]->(:Article) return ID(r)")
    rel_ids = [r[0] for r in out.values()]
    
rel_ids[:5]

[1095410, 1095409, 1095411, 1095412, 1095413]

In [185]:
q_merge_review_attributes = """MATCH p=(:Author)-[r:REVIEWS]->(:Article)
WHERE ID(r) = $rel_id
SET r.review_accepted = $review_accepted, r.review_text = $review_text
RETURN r
"""

created_review_rel_attr = []

random.seed(42)

with gdb.session() as session:
    for rel_id in rel_ids:
        _out = session.run(q_merge_review_attributes, 
                    rel_id=rel_id,
                    review_accepted=fake.pybool(), 
                    review_text=fake.texts(nb_texts=1, max_nb_chars=500)[0])
        created_review_rel_attr.append(_out.data())

In [186]:
created_review_rel_attr[0]

[{'r': <Relationship id=1095410 nodes=(<Node id=5860 labels=set() properties={}>, <Node id=46 labels=set() properties={}>) type='REVIEWS' properties={'review_text': 'Practice film world. Agency a ago partner force order true. Little reach national themselves free produce.\nMinute six either medical student some. Training affect half throughout maybe machine former.\nTend cold myself husband she. So half owner show water college cup. International white gun sport.\nPoor pretty create gun drug attention. Financial sense worker building the.', 'review_accepted': False}>}]

### Store a reviewing policy for each Journal or Conference

This can also be implemented as an `review_policy_min_count` to the `Journal` and `Proceedings` labels. We will set all of them to 3.

In [187]:
with gdb.session() as session:
    journal_ids = session.run("MERGE (n:Journal) SET n.review_policy_min_count = 3 return n").values()
    proceeding_ids = session.run("MERGE (n:Conference) SET n.review_policy_min_count = 3 return n").values()

In [188]:
journal_ids[:5]

[[<Node id=1489 labels={'Journal'} properties={'review_policy_min_count': 3, 'volume': '5', 'name': 'Evolutionary Ecology', 'pages': '118-127'}>],
 [<Node id=1499 labels={'Journal'} properties={'review_policy_min_count': 3, 'volume': '28 1', 'name': 'Emergency medicine Australasia : EMA', 'pages': '106-8'}>],
 [<Node id=1525 labels={'Journal'} properties={'review_policy_min_count': 3, 'volume': '35 6', 'name': 'The American journal of surgical pathology', 'pages': '904-12'}>],
 [<Node id=1529 labels={'Journal'} properties={'review_policy_min_count': 3, 'volume': '114', 'name': 'Human Genetics', 'pages': '280-283'}>],
 [<Node id=1537 labels={'Journal'} properties={'review_policy_min_count': 3, 'volume': '145 1', 'name': 'International journal of cardiology', 'pages': '142-6'}>]]

In [189]:
proceeding_ids[:2]

[[<Node id=8068 labels={'Conference'} properties={'review_policy_min_count': 3, 'name': 'Brian Murphy', 'id': 2}>],
 [<Node id=8085 labels={'Conference'} properties={'review_policy_min_count': 3, 'name': 'Leslie Wilson', 'id': 1}>]]

### Display the affiliation of the author to an organization or a company 

This can be solved by creating an `affiliation_institution_type` attribute, and also an `affiliation_institution_name`. Note that we could have created these as nodes, but the queries from part B do not refer to this information.


In [190]:
q_set_affiliations = """MATCH (n:Author)
WHERE n.id = $author_id
SET n.affiliation_institution_type = $affiliation_institution_type, n.affiliation_institution_name = $affiliation_institution_name
RETURN n
"""

institution_types = ['University', 'Company', 'NGO']
created_affiliations = []

random.seed(42)

with gdb.session() as session:
    for author_id in author_ids:
        _out = session.run(q_set_affiliations, 
                    author_id=author_id,
                    affiliation_institution_name=fake.company(), 
                    affiliation_institution_type=random.choice(institution_types))        
        created_affiliations.append(_out.values())